In [1]:
!pip install transformers einops accelerate bitsandbytes

In [2]:
from transformers import AutoTokenizer,AutoModelForSeq2SeqLM
from transformers import pipeline
import torch
import base64


In [3]:
checkpoint="MBZUAI/LaMini-T5-738M"

In [4]:
!pip install langchain

  Using cached langchain-0.1.15-py3-none-any.whl.metadata (13 kB)
  Using cached SQLAlchemy-2.0.29-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (9.6 kB)
  Using cached dataclasses_json-0.6.4-py3-none-any.whl.metadata (25 kB)
  Using cached jsonpatch-1.33-py2.py3-none-any.whl.metadata (3.0 kB)
  Using cached langchain_community-0.0.32-py3-none-any.whl.metadata (8.5 kB)
  Using cached langchain_core-0.1.41-py3-none-any.whl.metadata (5.9 kB)
  Using cached langchain_text_splitters-0.0.1-py3-none-any.whl.metadata (2.0 kB)
  Using cached langsmith-0.1.45-py3-none-any.whl.metadata (13 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.1/85.1 kB 15.3 MB/s eta 0:00:00
  Using cached marshmallow-3.21.1-py3-none-any.whl.metadata (7.2 kB)
  Using cached typing_inspect-0.9.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached packaging-23.2-py3-none-any.whl.metadata (3.2 kB)
  Using cached orjson-3.10.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (49 

In [6]:
from langchain.llms import HuggingFacePipeline

In [7]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
base_model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint, device_map="auto", torch_dtype=torch.float32)

/home/ec2-user/anaconda3/envs/JupyterSystemEnv/lib/python3.10/site-packages/torch/cuda/__init__.py:628: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [8]:
def llm_pipeline():
    pipe = pipeline(
        'text2text-generation',
        model=base_model,
        tokenizer=tokenizer,
        max_length=256,
        do_sample=True,
        temperature=0.3,
        top_p=0.95
    )
    local_llm = HuggingFacePipeline(pipeline=pipe)
    return local_llm

In [9]:
input_prompt="Write an article on AI"


In [10]:
model = llm_pipeline()
generated_text=model(input_prompt)
generated_text

/home/ec2-user/anaconda3/envs/JupyterSystemEnv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


'AI, or Artificial Intelligence, is a rapidly growing field that involves the development of computer systems that can perform tasks that typically require human intelligence. AI is a type of machine learning that involves training algorithms on large amounts of data to learn patterns and make predictions. The goal of AI is to create machines that can learn from experience, adapt to new situations, and make decisions based on that learning. AI is used in a variety of industries, including healthcare, finance, and transportation. Some of the most popular AI applications include speech recognition, image and video recognition, and autonomous vehicles. AI is used in a'

In [11]:
!pip uninstall -y sagemaker

Found existing installation: sagemaker 2.214.3
Uninstalling sagemaker-2.214.3:
  Successfully uninstalled sagemaker-2.214.3


In [12]:
!pip install sagemaker==2.168.0

  Using cached sagemaker-2.168.0-py2.py3-none-any.whl
  Using cached protobuf-3.20.3-cp310-cp310-manylinux_2_12_x86_64.manylinux2010_x86_64.whl.metadata (679 bytes)
  Using cached protobuf3_to_dict-0.1.5-py3-none-any.whl
  Using cached importlib_metadata-4.13.0-py3-none-any.whl.metadata (4.9 kB)
  Using cached PyYAML-6.0-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl.metadata (2.0 kB)
  Using cached tblib-1.7.0-py2.py3-none-any.whl.metadata (24 kB)
Using cached PyYAML-6.0-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (682 kB)
Using cached tblib-1.7.0-py2.py3-none-any.whl (12 kB)
Using cached importlib_metadata-4.13.0-py3-none-any.whl (23 kB)
Using cached protobuf-3.20.3-cp310-cp310-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (1.1 MB)
  Attempting uninstall: tblib
    Found existing installation: tblib 3.0.0
    Uninstalling tblib-3.0.0:
      Successfully uninstalled tblib-3.0.0
  

In [ ]:
!pip install protobuf==3.20.

In [14]:
import json
import sagemaker
import boto3
from sagemaker.huggingface import HuggingFaceModel, get_huggingface_llm_image_uri

try:
	role = sagemaker.get_execution_role()
except ValueError:
	iam = boto3.client('iam')
	role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

# Hub Model configuration. https://huggingface.co/models
hub = {
	'HF_MODEL_ID':'MBZUAI/LaMini-T5-738M',
    'HF_TASKS':'text2text-generation',
    'device_map':'auto',
    'torch_dtype':'torch.float32'
}

# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
	image_uri=get_huggingface_llm_image_uri("huggingface",version="0.8.2"),
	env=hub,
	role=role, 
)

# deploy model to SageMaker Inference
predictor = huggingface_model.deploy(
	initial_instance_count=1,
	instance_type="ml.g4dn.xlarge",
	container_startup_health_check_timeout=300,
  )
  
# send request
predictor.predict({
	"inputs": "Write an article on Count Sort",
})

-----------!

[{'generated_text': 'Count Sort is a popular algorithm used in data sorting that allows you to sort data '}]

In [24]:
payload={
    'inputs':input_prompt,
    'parameters':{
        'do_sample':True,
        'top_p':0.7,
        'top_k':50,
        'temperature':0.3,
        'max_new_tokens':512,
        'repition_penalty':1.03
    }
}


response=predictor.predict(payload)
print(response)

[{'generated_text': 'AI, or Artificial Intelligence, is a rapidly growing field that involves the development of computer systems that can perform tasks that typically require human intelligence, such as visual perception, speech recognition, decision-making, and language translation. AI has the potential to revolutionize various industries, from healthcare to finance, and has the potential to revolutionize the way we live and work. One of the most significant benefits of AI is its ability to automate repetitive tasks, freeing up time for more complex and creative tasks. AI algorithms can analyze large amounts of data quickly and accurately, allowing for more efficient and accurate decision-making.'}]


In [25]:
END_POINT="huggingface-pytorch-tgi-inference-0000-00-00-00-00-00-000"

In [26]:
import boto3

In [27]:
runtime=boto3.client('runtime.sagemaker')

In [28]:
response=runtime.invoke_endpoint(EndpointName=END_POINT,ContentType="application/json",Body=json.dumps(payload))

In [29]:
print(response)

{'ResponseMetadata': {'RequestId': 'eba98174-cfde-4c2c-bba9-8fe95ba13104', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'eba98174-cfde-4c2c-bba9-8fe95ba13104', 'x-amzn-invoked-production-variant': 'AllTraffic', 'date': 'Thu, 11 Apr 2024 12:07:56 GMT', 'content-type': 'application/json', 'content-length': '688', 'connection': 'keep-alive'}, 'RetryAttempts': 0}, 'ContentType': 'application/json', 'InvokedProductionVariant': 'AllTraffic', 'Body': <botocore.response.StreamingBody object at 0x7f1af483b370>}


In [30]:
prediction=json.loads(response['Body'].read().decode('utf-8'))

In [32]:
prediction[0]['generated_text']

'AI, or Artificial Intelligence, is a rapidly growing field that aims to create intelligent machines that can perform tasks that typically require human intelligence. AI is a subset of computer science that focuses on creating machines that can think, reason, and learn from data. AI is used in a variety of applications, including image and speech recognition, natural language processing, and predictive analytics. AI is used in various industries, including healthcare, finance, and transportation. It has the potential to revolutionize the way we live and work, and it has the potential to revolutionize industries such as healthcare, finance, and transportation'